In [22]:
import ipywidgets as widgets
from groq import Groq
from IPython.display import display

In [23]:
def build_prompt(topic: str, duration: int, emotion: str = "formal", audience: str = "general") -> str:
    """
    Build a detailed prompt for Groq-based speech generation.
    """
    style_templates = {
        "formal": "Write a formal {duration}-minute speech about '{topic}' suitable for a professional audience.",
        "casual": "Write a casual, friendly {duration}-minute speech about '{topic}'.",
        "motivational": "Write an inspiring {duration}-minute motivational speech about '{topic}' that energizes the audience.",
        "persuasive": "Write a compelling {duration}-minute persuasive speech about '{topic}' to change minds.",
        "instructional": "Write a step-by-step {duration}-minute instructional speech on '{topic}'.",
        "debate": "Write a {duration}-minute debate speech about '{topic}' with strong arguments and counterpoints.",
    }

    audience_guidance = {
        "general": "Make the speech accessible to a general audience with no specialized knowledge.",
        "experts": "Include technical depth suitable for experts in the field.",
        "children": "Use simple, engaging language and examples suitable for kids.",
        "students": "Be educational and engaging for a student audience.",
        "executives": "Focus on strategic implications and leadership perspectives.",
    }

    base_prompt = style_templates.get(emotion, style_templates["formal"]).format(topic=topic, duration=duration)
    audience_note = audience_guidance.get(audience, "")
    
    final_prompt = (
        f"{base_prompt}\n\n"
        f"{audience_note}\n\n"
        f"Structure the speech with an introduction, body, and conclusion.\n"
        f"Use engaging transitions, rhetorical devices, and paragraph breaks.\n"
        f"Aim for roughly {duration * 130} words."
    )
    
    return final_prompt

In [24]:
def generate_speech(topic: str, duration: int, emotion: str, audience: str):
    # Build prompt based on input
    prompt = build_prompt(topic, duration, emotion, audience)
    
    # Groq API call
    api_key = "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"
    client = Groq(api_key=api_key)
    completion = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.9,
        max_tokens=2048,
        top_p=1,
        stream=True
    )
    
    # Collect and display streaming output
    speech = ""
    for chunk in completion:
        speech += chunk.choices[0].delta.content or ""
    return speech


In [25]:
topic_input = widgets.Text(value="climate change", description="Topic:")
emotion_dropdown = widgets.Dropdown(
    options=["formal", "casual", "motivational", "persuasive", "instructional", "debate"],
    value="motivational",
    description="Emotion:"
)
duration_slider = widgets.IntSlider(value=5, min=1, max=30, step=1, description="Duration (min):")
audience_dropdown = widgets.Dropdown(
    options=["general", "experts", "children", "students", "executives"],
    value="general",
    description="Audience:"
)


In [27]:
# --- Display the Widgets ---
display(topic_input, emotion_dropdown, duration_slider, audience_dropdown)

# --- Button to Generate Speech ---
generate_button = widgets.Button(description="Generate Speech")
output = widgets.Output()
def on_button_click(b):
    with output:
        print("Generating speech...\n")
        # Call the function to generate speech
        speech = generate_speech(topic_input.value, duration_slider.value, emotion_dropdown.value, audience_dropdown.value)
        print(speech)
        # Save the generated speech to a file
        with open("generated_speech.txt", "w") as file:
            file.write(speech)
        
        print("Speech successfully generated and saved to 'generated_speech.txt'.")
generate_button.on_click(on_button_click)
display(generate_button, output)

Text(value='VIJAY as politician', description='Topic:')

Dropdown(description='Emotion:', index=1, options=('formal', 'casual', 'motivational', 'persuasive', 'instruct…

IntSlider(value=30, description='Duration (min):', max=30, min=1)

Dropdown(description='Audience:', index=2, options=('general', 'experts', 'children', 'students', 'executives'…

Button(description='Generate Speech', style=ButtonStyle())

Output()

In [21]:
with open("generated_speech.txt", "w") as file:
        file.write(speech)

print("Speech successfully generated and saved to 'generated_speech.txt'.")

NameError: name 'speech' is not defined